In [ ]:
# 이 코랩파일은 유튜브 https://www.youtube.com/watch?v=lSBX-nMQ8cE 에 올라온 Gradio를 이용한 Llama2 프롬프팅 가이드 ipynb 파일을 일부 수정한 파일(코드)입니다.

# 이 파일에서 수정된 부분은 다음과 같습니다.
# 1. road llm model 변경: Llama2 -> Fine-tuning Gemma-2B   (https://huggingface.co/taeseo06/Gemma-2b-PersonalizedFinetuning)
# 2. 모델을 불러오는 과정에서 GoogleDrive 를 연결하여 드라이브에 저장된 파인튜닝 모델, 토크나이저 직접 로드하는 방식 차용
# 3. 한국어 답변 및 일부 출력 오류를 해결하기 위해 generate(모델 생성)부분 일부 수정

In [1]:
!pip install transformers torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.9 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl s

In [ ]:
# import locale
# locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [4]:
!huggingface-cli whoami

taeseo06


 ---
## 모델, 토크나이저 로드
GoogleDrive를 연결하여 직접 드라이브에 접근해서 파인튜닝 된 모델과 토크나이저를 불러오는 방법과, HuggingFace Pipeline으로 모델을 로드하는 방법으로 나누었습니다.

! 외부에서 시연 및 접근 할 때에는 GoogleDrive 연결이 아닌 HuggingFace 로드가 적합.


### 1️⃣. GoogleDrive에서 직접적으로 모델, 토크나이저 로드

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#@title TrainedModel 폴더에 저장된 파인튜닝 모델이름 -> 시도 횟수
trial = '3' #@param {type: "string"}


In [9]:
# 현재 학습을 시도하는 횟수를 지정해서 각각의 경로마다 변경할 수 있게
Route = '/content/drive/MyDrive/Colab Notebooks/LLM 파인튜닝 프로젝트 (model-llama2)/llama2-PersonalizedFinetuning/TrainedModel/' + str(trial) + 'st' # 학습된 모델의 기본 루트

%cd $Route

/content/drive/MyDrive/Colab Notebooks/LLM 파인튜닝 프로젝트 (model-llama2)/llama2-PersonalizedFinetuning/TrainedModel/3st


In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("./merged_LLM_Model")
tokenizer = AutoTokenizer.from_pretrained("./merged_LLM_Model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 2️⃣. HuggingFace에 Pipeline 으로 모델, 토크나이저 로드

In [31]:
from transformers import AutoTokenizer, pipeline
import transformers
import torch

model = "taeseo06/Gemma-2b-PersonalizedFinetuning" # https://huggingface.co/taeseo06/Gemma-2b-PersonalizedFinetuning


gemma_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


 ---
 ## Generate - 모델의 생성 함수

In [15]:
def get_response(prompt: str) -> None:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = gemma_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=512,
    )
    print("Chatbot:", sequences[0]['generated_text'])



In [28]:
get_response("안녕")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Chatbot: 안녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕


In [30]:
get_response("선린인터넷고등학교의 위치를 알려줘")

Chatbot: 선린인터넷고등학교의 위치를 알려줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘


In [32]:
# 텍스트 생성 함수 정의
def generate_text(prompt, max_length=512):
  # 토크나이징
  input_ids = tokenizer.encode(prompt, return_tensors="pt")

  # 모델 추론
  outputs = model.generate(input_ids=input_ids, max_length=max_length)

  # 토큰을 텍스트로 변환
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return generated_text

# 예시
# prompt = "문태서라는 인물에 관해서 설명해줘"
# generated_text = generate_text(prompt)
# print(f"Generated text: {generated_text}")


In [23]:
prompt = "안녕"
generated_text = generate_text(prompt)
print(f"Generated text: {generated_text}")

Generated text: 안녕하세요!

I'm looking for a fun and engaging way to learn Korean, and I've heard that watching Korean dramas is a great way to do that. But I'm not sure where to start.

What are some good places to watch Korean dramas?

I'd also like to know if there are any specific recommendations for beginner viewers.

Thank you for your help!


In [26]:
prompt = "한국어로 답변할 수 있어?"
generated_text = generate_text(prompt)
print(f"Generated text: {generated_text}")

Generated text: 한국어로 답변할 수 있어?

Sure, here's the answer in Korean:

Sure, here's the answer in Korean:

**자신의 꿈과 목표를 정하고, 이를 달성하기 위해 어떻게 해야 할지 탐구해 보세요.**


### Drawbacks of `get_response()`

1. **Lack of Conversation History**: The basic approach does not account for past interactions, making it less effective for maintaining a coherent conversation.
2. **Limited Customization**: The function doesn't allow for advanced prompt formatting or handling system-level instructions.
3. **Not Ready for UI Integration**: This basic approach isn't designed for easy integration with user interface libraries like Gradio.

## Improved Prompts

The right structure of Llama 2 prompts:

```
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>

{{ user_message }} [/INST]
```

In [ ]:
# Gemma


'''
<bos><start_of_turn>user
Write a hello world program<end_of_turn>
<start_of_turn>model
'''

### Building the Prompt

Explaining the parameters:
- `message` is the current message we're sending
- `history` is the history of conversation as a list of tupples `[(user_msg1, bot_msg1), (usr_msg2, bot_msg2), ...]`


In [ ]:
SYSTEM_PROMPT = """<s>[INST] <<SYS>>
You are a helpful bot. Your answers are clear and concise.
<</SYS>>

"""

SYSTEM_PROMPT = """

"""




# Formatting function for message and history
def format_message(message: str, history: list, memory_limit: int = 3) -> str:
    """
    Formats the message and history for the Llama model.

    Parameters:
        message (str): Current message to send.
        history (list): Past conversation history.
        memory_limit (int): Limit on how many past interactions to consider.

    Returns:
        str: Formatted message string
    """
    # always keep len(history) <= memory_limit
    if len(history) > memory_limit:
        history = history[-memory_limit:]

    if len(history) == 0:
        return SYSTEM_PROMPT + f"{message} [/INST]"

    formatted_message = SYSTEM_PROMPT + f"{history[0][0]} [/INST] {history[0][1]} </s>"

    # Handle conversation history
    for user_msg, model_answer in history[1:]:
        formatted_message += f"<s>[INST] {user_msg} [/INST] {model_answer} </s>"

    # Handle the current message
    formatted_message += f"<s>[INST] {message} [/INST]"

    return formatted_message

### Getting Responses

We need the function to generate responses.

In [ ]:
# Generate a response from the Llama model
def get_llama_response(message: str, history: list) -> str:
    """
    Generates a conversational response from the Llama model.

    Parameters:
        message (str): User's input message.
        history (list): Past conversation history.

    Returns:
        str: Generated response from the Llama model.
    """
    query = format_message(message, history)
    response = ""

    sequences = llama_pipeline(
        query,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1024,
    )

    # generated_text = sequences[0]['generated_text']
    generated_text = generate_text(query)
    response = generated_text[len(query):]  # Remove the prompt from the output

    print("Chatbot:", response.strip())
    return response.strip()



# message = "안녕"

# query = format_message(message, [])
# response = ""

# sequences = llama_pipeline(
#         query,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id,
#         max_length=1024,
#     )

# # generated_text = sequences[0]['generated_text']

# generated_text = generate_text(query)
# response = generated_text[len(query):]  # Remove the prompt from the output

# print("Chatbot:", response.strip())


In [ ]:
response

'\n\nI am a helpful bot. I am here to assist you with any questions or tasks you may have. How can I help you today?'

In [ ]:
response.strip()

'I am a helpful bot. I am here to assist you with any questions or tasks you may have. How can I help you today?'

In [ ]:
import gradio as gr

gr.ChatInterface(get_llama_response).launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://42dcfbe4df95c69f26.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Chatbot: 님! 어떻게 되세요?

안녕하세요! 저는 어떻게 되세요? 질문해 주세요.
Chatbot: 
Chatbot: 
Chatbot: 


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1689, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1253, in call_function
    prediction = await fn(*processed_input)
  File "/usr/local/lib/python3.10/dist-packages/gradio/utils.py", line 717, in async_wrapper
    response = await f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/gradio/chat_interface.py", line 504, in _submit_fn
    response = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_async

### Conclusion

Thanks to the Hugging Face Library, creating a pipeline to chat with llama 2 (or any other open-source LLM) is quite easy.

But if you worked a lot with much larger models such as GPT-4, you need to adjust your expectations.